In [ ]:
import tensorflow as tf 
from tensorflow import keras 
import tensorflow_addons as tfa
import tensorflow_hub as hub 
from sklearn.metrics import mean_absolute_error 
import numpy as np 
import datetime

c:\Users\21265\anaconda3\envs\tfpy39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Load data

In [ ]:
train_ds = tf.data.experimental.load('./data/faces/train_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

valid_ds = tf.data.experimental.load('./data/faces/valid_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds, valid_ds

(<PrefetchDataset shapes: ((None, 10, 224, 224, 3), (None, 5)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 10, 224, 224, 3), (None, 5)), types: (tf.float32, tf.float32)>)

### Load VGG16

In [ ]:
vgg16 = keras.applications.VGG16(include_top=False, weights='imagenet', pooling='max')
vgg16.trainable = False

### Load Vit-B16

In [ ]:
vit_extractor = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-fe/versions/1/", trainable=False) # ../../vit_b16_fe/

### Build Face model

In [ ]:
inputs = keras.layers.Input(shape=(10,224,224,3), name='Input')

# First model 
x      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./255.0), name='Rescaling')(inputs)
x      = keras.layers.TimeDistributed(vgg16, name='vgg16')(x)
x      = keras.layers.LSTM(units=128, return_sequences=True)(x)
x      = keras.layers.LSTM(units=64)(x)
x      = keras.layers.Dropout(0.2)(x)

x      = keras.layers.Dense(units=1024)(x)
x      = keras.layers.Dense(units=512, activation='relu')(x)


# Second model 
y      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./127.5, offset=-1))(inputs)

# vit_extractor = hub.KerasLayer("../../vit_b16_fe/", trainable=False) # ../../vit_b16_fe/
vit_extractor.compute_output_shape = lambda x: (x[0], 768)

y      = keras.layers.TimeDistributed(vit_extractor)(y)
y      = keras.layers.LSTM(units=128, return_sequences=True)(y)
y      = keras.layers.LSTM(units=64)(y)
y      = keras.layers.Dropout(0.2)(y)
y      = keras.layers.Dense(units=1024)(y)
y      = keras.layers.Dense(units=512, activation='relu')(y)


# Averaget two models
z      = keras.layers.Average()([x,y])

z      = keras.layers.Dense(256, activation='relu')(z)
z      = keras.layers.Dropout(0.5)(z)

z      = keras.layers.Dense(5, activation='sigmoid')(z)

model  = keras.models.Model(inputs=inputs, outputs=z)
keras.utils.plot_model(model, show_shapes=True)

### Compile model

In [ ]:
t = datetime.datetime.now().strftime("%m%d_%H%M%S")

optimizer      = tfa.optimizers.RectifiedAdam()

early_stopping = keras.callbacks.EarlyStopping(patience=10, verbose=0)
check_point    = keras.callbacks.ModelCheckpoint(filepath='./weights/face/'+str(t)+'/face.t5',
                             monitor='val_mae',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=0)

model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

### Train

In [ ]:
history = model.fit(train_ds, validation_data=valid_ds, batch_size=32, epochs=100, callbacks=[early_stopping, check_point])

Epoch 1/100
188/188 [==============================] - 357s 1s/step - loss: 0.0199 - mae: 0.1128 - val_loss: 0.0166 - val_mae: 0.1030
Epoch 2/100
188/188 [==============================] - 256s 1s/step - loss: 0.0168 - mae: 0.1040 - val_loss: 0.0176 - val_mae: 0.1062
Epoch 3/100
188/188 [==============================] - 240s 1s/step - loss: 0.0157 - mae: 0.1004 - val_loss: 0.0152 - val_mae: 0.0986
Epoch 4/100
188/188 [==============================] - 214s 1s/step - loss: 0.0148 - mae: 0.0971 - val_loss: 0.0151 - val_mae: 0.0978
Epoch 5/100
188/188 [==============================] - 214s 1s/step - loss: 0.0144 - mae: 0.0958 - val_loss: 0.0149 - val_mae: 0.0970
Epoch 6/100
188/188 [==============================] - 214s 1s/step - loss: 0.0136 - mae: 0.0932 - val_loss: 0.0160 - val_mae: 0.1013
Epoch 7/100
188/188 [==============================] - 213s 1s/step - loss: 0.0131 - mae: 0.0916 - val_loss: 0.0134 - val_mae: 0.0920
Epoch 8/100
188/188 [==============================] - 213s 1s

### Load wights

In [9]:
model.load_weights('./weights/face/0225_154249/face.t5')

## Evaluation

### Validation data

In [ ]:
valid_ds = tf.data.experimental.load('./data/faces/valid_ds') 
valid_ds

<_LoadDataset shapes: ((None, 10, 224, 224, 3), (None, 5)), types: (tf.float32, tf.float32)>

In [10]:
loss, mae = model.evaluate(valid_ds)
(1-mae)*100

63/63 [==============================] - 117s 1s/step - loss: 0.0123 - mae: 0.0881


91.19105786085129

In [11]:
y_true = np.concatenate([y for x,y in valid_ds])
y_pred = model.predict(valid_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

(array([91.122856, 91.51319 , 91.2349  , 91.382095, 90.702225],
       dtype=float32),
 91.19105562567711)

### Test data

In [12]:
test_ds = tf.data.experimental.load('./data/faces/test_ds/')
loss, mae = model.evaluate(test_ds)
(1-mae)*100

63/63 [==============================] - 62s 974ms/step - loss: 0.0127 - mae: 0.0895


91.05187579989433

In [13]:
y_true = np.concatenate([y for x,y in test_ds])
y_pred = model.predict(test_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

(array([90.81782 , 91.483826, 91.21508 , 91.05965 , 90.683   ],
       dtype=float32),
 91.05187132954597)